In [ ]:
!pip install youtube-transcript-api
!pip install google-api-python-client

To get list of all the video urls on a yt channel

In [30]:
### search for externalId in the source code to find channel id
from googleapiclient.discovery import build

def get_video_urls(channel_id, api_key):
    youtube = build('youtube', 'v3', developerKey=api_key)
    video_urls = []

    # Fetch the uploads playlist ID for the channel
    request = youtube.channels().list(part='contentDetails', id=channel_id)
    response = request.execute()
    playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Fetch all videos in the uploads playlist
    next_page_token = None
    while True:
        pl_request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        pl_response = pl_request.execute()

        video_ids = [item['contentDetails']['videoId'] for item in pl_response['items']]
        video_urls.extend([f'https://www.youtube.com/watch?v={id}' for id in video_ids])

        next_page_token = pl_response.get('nextPageToken')

        if not next_page_token:
            break

    return video_urls

def save_urls_to_file(urls, filename):
    with open(filename, 'w') as file:
        for url in urls:
            file.write(url + '\n')

# Replace with your API key and the channel ID you're interested in
api_key = ''
channel_id = ''
filename = 'file1.txt'

video_urls = get_video_urls(channel_id, api_key)
save_urls_to_file(video_urls, filename)


To get transcripts of videos

In [31]:
import os
from typing import List
from pathlib import Path
from urllib.parse import urlparse, parse_qs

# pip install youtube-transcript-api
from youtube_transcript_api import YouTubeTranscriptApi

def read_urls_from_file(file_path: Path) -> List[str]:
    """
    Reads URLs from a text file and returns them as a list.
    """
    with open(file_path, 'r') as file:
        return file.read().splitlines()

def get_youtube_video_id_from_url(url: str) -> str:
    """
    Extracts and returns the video ID from the given YouTube URL.
    """
    query_string = urlparse(url).query
    parameters = parse_qs(query_string)
    return parameters.get('v', [None])[0]

def get_youtube_video_transcript(video_id: str) -> str:
    """
    Returns transcript of the given 'video_id'.
    """
    try:
        transcript = YouTubeTranscriptApi.get_transcript(
            video_id, languages=['en-US', 'en']
        )
        utterances = [p['text'] for p in transcript]
        return ' '.join(utterances)

    except Exception as e:
        return "\nCould not retrieve a transcript: " + str(e)  # Return the exception message with a prefix

def save_transcript(transcript: str, video_id: str, directory: Path):
    """
    Stores the transcript locally in a text file named after the video ID.
    """
    file_path = directory / f'{video_id}.txt'  # Create file path with video ID

    # Skip saving if the transcript starts with the error message
    if transcript.startswith("\n"):
        return

    if transcript is not None:
        with open(file_path, 'w') as output_file:
            output_file.write(transcript)

# Specify the directory for saving the transcripts and the file containing the URLs
urls_file_path = Path('/content/file1.txt') # Replace with the path to your file
output_directory = Path('/content/data') # Specify the directory for saving transcripts

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

urls = read_urls_from_file(urls_file_path)

for url in urls:
    video_id = get_youtube_video_id_from_url(url)

    if video_id:
        transcript = get_youtube_video_transcript(video_id)
        save_transcript(transcript, video_id, output_directory)
    else:
        print(f"Invalid YouTube URL: {url}")


To zip the folder

In [35]:
!zip -r /content/MarkBell.zip /content/MarkBell

To delete the folder

In [34]:
import shutil

shutil.rmtree('/content/MarkBell', ignore_errors=True)
